# YOLOv8 Finetuning for Object Detection and Bounding Boxes Problems

In this notebook, we fine-tuned a YOLO model for object detection using a custom training dataset, then used the fine-tuned model to make predictions on the test set and generate a submission file for Kaggle.

In [ ]:
## STEP 0: IMPORTING NEEDED LIBRARIES

%pip install -q ultralytics

from ultralytics import YOLO
import pandas as pd
import os
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import torch
import yaml

# use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
## STEP 1: DATA PREPARATION

# paths
train_csv = "./train.csv"
image_root = "./input/data-bounty-6-product-object-detection" # path to the train dataset
output_dir = "./output/yolo_dataset"  # path to save the YOLO dataset
train_images_dir = f"{output_dir}/images/train"
train_labels_dir = f"{output_dir}/labels/train"
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)

# load train
df = pd.read_csv(train_csv)

# encode classes
all_classes = []
for row in df['prediction_string']:
    parts = row.split()
    all_classes.extend(parts[::6])
label_encoder = LabelEncoder()
label_encoder.fit(all_classes)
class2id = {name: idx for idx, name in enumerate(label_encoder.classes_)}
id2class = {idx: name for name, idx in class2id.items()}

# save class names
class_names_path = f"{output_dir}/class_list.txt"
with open(class_names_path, "w") as f:
    f.write("\n".join(label_encoder.classes_))

# convert XYXY to YOLO format
def convert_bbox(x1, y1, x2, y2, img_w, img_h):
    x = (x1 + x2) / 2 / img_w
    y = (y1 + y2) / 2 / img_h
    w = (x2 - x1) / img_w
    h = (y2 - y1) / img_h
    return x, y, w, h

# create YOLO labels
for _, row in tqdm(df.iterrows(), total=len(df)):
    image_id = row["image_id"]
    image_path = f"{image_root}/Train/JPEGImages/{image_id}.jpg"
    label_path = f"{train_labels_dir}/{image_id}.txt"
    os.system(f"cp {image_path} {train_images_dir}/{image_id}.jpg")

    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    with open(label_path, "w") as f:
        items = row["prediction_string"].split()
        for i in range(0, len(items), 6):
            class_name = items[i]
            x1, y1, x2, y2 = map(int, items[i+2:i+6])
            class_id = class2id[class_name]
            x, y, bw, bh = convert_bbox(x1, y1, x2, y2, w, h)
            f.write(f"{class_id} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}\n")

# create YOLOv8 data.yaml
data_yaml = {
    "train": train_images_dir,
    "val": train_images_dir,
    "nc": len(label_encoder.classes_),
    "names": [str(name) for name in label_encoder.classes_.tolist()]  # <- fix here
}

data_yaml_path = f"{output_dir}/data.yaml"
with open(data_yaml_path, "w") as f:
    yaml.dump(data_yaml, f)

In [ ]:

# STEP 2: FINETUNING THE MODEL

model = YOLO("./best.pt")
results = model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=8,
    device=device,
    workers=2,
    project="/kaggle/working/",
    name="yolo_finetuned"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 21.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 whic

100%|██████████| 3796/3796 [01:16<00:00, 49.93it/s]


Ultralytics 8.3.116 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/yolo/other/default/1/best.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=2, project=/kaggle/working/, name=yolo_finetuned, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

100%|██████████| 755k/755k [00:00<00:00, 24.4MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.conv.Conv             [640

...

                   all       3795      18992      0.998      0.998      0.995      0.985

5 epochs completed in 0.638 hours.
Optimizer stripped from /kaggle/working/yolo_finetuned/weights/last.pt, 136.9MB
Optimizer stripped from /kaggle/working/yolo_finetuned/weights/best.pt, 136.9MB

Validating /kaggle/working/yolo_finetuned/weights/best.pt...
Ultralytics 8.3.116 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 68,232,387 parameters, 0 gradients, 258.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 238/238 [01:46<00:00,  2.23it/s]


                   all       3795      18992      0.998      0.998      0.995      0.985
                 3+2-2         44        126      0.998          1      0.995      0.975
                 3jia2         46        124          1      0.997      0.995       0.98
              aerbeisi         22         38      0.997          1      0.995      0.988
                anmuxi         70        107      0.999      0.991      0.995      0.972
                aoliao         56        613          1      0.995      0.995      0.942
                 asamu         88        246          1          1      0.995      0.991
                baicha         47        107      0.987          1      0.992      0.981
            baishikele         70        160          1      0.994      0.995      0.984
          baishikele-2         44         75      0.995          1      0.995      0.986
            baokuangli         59        179      0.999          1      0.995       0.99
           binghongch

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 24.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/yolo_finetuned


In [ ]:
## STEP 3: INFERENCE ON TEST DATASET

test_images = sorted(glob(f"{image_root}/Test/JPEGImages/*.jpg"))
conf_threshold = 0.4
iou_threshold = 0.4
final_model = YOLO("/kaggle/working/yolo_finetuned/weights/best.pt")

submission_data = []

for img_path in tqdm(test_images):
    result = final_model.predict(
        source=img_path,
        conf=conf_threshold,
        iou=iou_threshold,
        device=device,  # ✅ GPU if available
        verbose=False
    )[0]

    image_id = os.path.splitext(os.path.basename(img_path))[0]
    prediction_string = ""

    for box in result.boxes:
        cls_id = int(box.cls.item())
        score = box.conf.item()
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        prediction_string += f"{id2class[cls_id]} {score:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)} "

    submission_data.append({
        "image_id": image_id,
        "prediction_string": prediction_string.strip()
    })

100%|██████████| 542/542 [00:34<00:00, 15.52it/s]


✅ Final submission saved to /kaggle/working/submission10.csv


In [ ]:
## STEP 4: SUBMISSION FILE

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv("./output/submission.csv", index=False)
submission_df.head()

,image_id,prediction_string
0,XYG2020121711403780263182_81,damaicha 0.9782 540 109 631 194 damaicha 0.976...
1,XYG2020121811045959626219_81,youlemei 0.9773 286 151 367 227 hongniu2 0.977...
2,XYG2020121812002586066796_81,baicha 0.9797 299 120 401 225 moliqingcha 0.97...
3,XYG2020122214431356849310_81,yida 0.9795 454 431 513 489 damaicha 0.9790 65...
4,XYG2020122214522128735418_81,yida 0.9765 440 313 504 374 baishikele-2 0.976...
